In [1]:
# from re import S
import tensorflow as tf

from VideoSwinTransformer import SwinTransformer3D , SwinTransformer3D_pt 


swin = SwinTransformer3D()



x = tf.random.uniform(
    [1,8,32,32,3],
    minval=0,
    maxval=255,
    dtype=tf.dtypes.float32,
    name="x"
)

2022-08-05 07:46:28.922789: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-05 07:46:29.579813: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10792 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


In [1]:
print("x")

x


In [2]:
! pip install  -r requirements.txt

In [3]:
!pip install timm

In [2]:
import tensorflow as tf
import torch
from VideoSwinTransformer import model_configs , SwinTransformer3D_pt
from collections import OrderedDict








In [3]:
tf_model = tf.keras.models.load_model('swin_tiny_patch244_window877_kinetics400_1k_tf.pth')

In [4]:
x_pt = torch.rand( (1,3, 8,224,224)) * 255
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))
x_tf = tf.cast(x_tf, dtype= tf.float32)

x_tf.shape

TensorShape([1, 8, 224, 224, 3])

In [10]:
# x = tf.random.uniform(
#     [1,32,32,32,3],
#     minval=0,
#     maxval=255,
#     dtype=tf.dtypes.float32,
#     name="x"
# )

In [5]:
y = tf_model.predict(x_tf)

2022-08-05 07:47:26.378015: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


In [6]:
y.shape

(1, 768, 4, 7, 7)

In [7]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']

pt_model = SwinTransformer3D_pt(**cfg)

checkpoint = torch.load(f'{name}.pth')


new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

In [8]:
z = pt_model(x_pt)

In [9]:

z.shape


torch.Size([1, 768, 4, 7, 7])

In [10]:
multiplier = 1000
y = tf.round(y * multiplier) / multiplier

In [11]:
y[0][0][0]

<tf.Tensor: shape=(7, 7), dtype=float32, numpy=
array([[ 0.045,  0.344,  0.253, -0.004,  0.009,  0.705,  0.26 ],
       [ 0.136,  0.811,  0.709,  0.256,  0.876,  1.175, -0.088],
       [ 0.265,  0.203,  0.006,  0.275,  0.102,  0.439,  0.102],
       [-0.039,  0.128, -0.076,  0.018,  0.043, -0.098, -0.034],
       [ 0.053,  0.055, -0.054,  0.221,  0.296,  0.589, -0.039],
       [ 0.257,  0.86 ,  0.119,  0.043,  0.269,  0.301,  0.04 ],
       [ 0.289,  0.674, -0.081,  0.258,  0.264, -0.006,  0.245]],
      dtype=float32)>

In [12]:
z[0][0][0]

tensor([[-0.0353,  0.3393,  0.2231,  0.0271,  0.0512,  0.5188,  0.2515],
        [ 0.0039,  0.6314,  0.5279,  0.2193,  0.6839,  0.9365, -0.0378],
        [ 0.2619,  0.2776,  0.0552,  0.2433,  0.1299,  0.1143, -0.0433],
        [-0.0142,  0.0622, -0.0502, -0.0265,  0.1082, -0.0870, -0.0278],
        [ 0.3037,  0.2018,  0.0189,  0.0455,  0.1578,  0.7265,  0.0630],
        [ 0.2615,  0.7248,  0.0369,  0.0627,  0.2571,  0.3040,  0.1014],
        [ 0.2778,  0.5499, -0.0418,  0.2637,  0.2586, -0.0115, -0.0399]],
       grad_fn=<SelectBackward0>)

In [ ]:
model = tf.keras.Sequential()
model.add(SwinTransformer3D())
model.add(tf.keras.layers.Dense(6))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=10, epochs=10)

In [13]:
x = tf.random.normal([100,8, 32,32, 3])
y = tf.random.uniform(shape=[100], minval=0, maxval=5, dtype='int64')
y

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([1.5589648 , 4.131706  , 3.424728  , 0.03354549, 3.9374757 ,
       1.9532555 , 1.4631528 , 4.9608464 , 4.790521  , 2.7811658 ,
       0.8233148 , 0.67228556, 0.66149294, 2.6740491 , 2.8545089 ,
       2.5485253 , 2.4126172 , 0.77902675, 1.8516135 , 2.4605334 ,
       2.8350801 , 1.038543  , 0.9111941 , 4.994187  , 1.8475294 ,
       1.8963927 , 3.8615584 , 3.4105873 , 1.9966418 , 3.9203568 ,
       3.3940077 , 3.6697948 , 2.760222  , 0.54742575, 3.2439911 ,
       4.9453897 , 4.101827  , 3.5235178 , 4.7893124 , 0.11487126,
       4.6799335 , 3.256632  , 1.5831792 , 0.0055778 , 4.6060953 ,
       1.911403  , 3.8623018 , 4.57571   , 2.8755665 , 3.96671   ,
       2.1448815 , 0.95594823, 3.726253  , 2.0881004 , 4.0867558 ,
       1.0058558 , 3.2285786 , 0.82421184, 2.2420616 , 3.028944  ,
       4.080575  , 2.0648265 , 1.316492  , 0.95437944, 3.8159304 ,
       4.1701217 , 2.4723864 , 3.4331596 , 1.501295  , 1.3364708 ,
       4.16945

In [ ]:
x  = swin(x)
#print(x.shape)

In [14]:
# swin.summary()

In [3]:
tf.constant(-100, dtype= 'float64')

<tf.Tensor: shape=(), dtype=float64, numpy=-100.0>